[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ACS-IC-labs/IC-labs/blob/main/labs/lab07/lab7.ipynb)

# Laboratorul 07 - Correlation Power Analysis

#### Autori: Marios Choudary, Răzvan Smădu

În acest laborator vom implementa un atac de tip side-channel denumit **Correlation Power Analysis**.

Pașii pentru realizarea atacului sunt următorii:

1. Se alege un target din algoritm pentru a fi atacat (în general output-ul unui S-box folosit de un block cipher – în cazul nostru fiind AES)
2. Se obține un număr mare de sample-uri de leakage de la target când acesta procesează diferite plaintext-uri
3. Se alege un model de leakage potrivit (de obicei hamming weight pentru valoarea target)
4. Pentru fiecare cheie posibilă (ex. toate valorile de la 0 la 255), se calculează [Pearson's correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) dintre modelul estimat de leakage pentru cheia curentă și trace-urile de leakage
5. Se păstrează cheia pentru care se obține corelația maximă

Mai multe detalii vor fi prezentate la laborator. Prezentarea PowerPoint pentru acest laborator poate fi găsită [aici](https://drive.google.com/file/d/1GqRveIHLY6MxtXjedKS-X4m3RbXQypDD/view).

## Setup

Vom folosi biblioteca [NumPy](https://numpy.org/doc/stable/) pentru realizarea de operații pe matrice în mod eficient și mult mai rapid decât cu funcțiile built-in din Python. Pentru realizarea graficelor, vom folosi [matplotlib](https://matplotlib.org/stable/index.html) care are un API foarte asemanator cu cel din MATLAB/Octave.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Ne vom folosi de S-Box-ul din AES precalculat și de funcția pentru calcularea Hamming Weight (i.e., numărul de 1 din reprezentarea binară).

In [ ]:
# Rijndael S-box
s_box = np.array([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01,
                  0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d,
                  0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4,
                  0x72, 0xc0, 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc,
                  0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7,
                  0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2,
                  0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e,
                  0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
                  0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb,
                  0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 0xfb,
                  0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
                  0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5,
                  0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c,
                  0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d,
                  0x64, 0x5d, 0x19, 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a,
                  0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
                  0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3,
                  0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 0xe7, 0xc8, 0x37, 0x6d,
                  0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a,
                  0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6,
                  0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 0x3e,
                  0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
                  0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9,
                  0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
                  0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99,
                  0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16])


def hamming_weight(X: np.ndarray) -> np.ndarray:
    """Computes the Hamming weight

    Parameters
    ----------
    X : np.ndarray
        A numpy array or matrix of integer elements.

    Returns
    -------
    np.ndarray
        The Hamming weight for each element from X.
    """
    assert X.dtype in [np.int8, np.int16, np.int32, np.int64], \
        "Expected integer values, but provided %s" % X.dtype
    return np.vectorize(lambda x: bin(x).count("1"))(X)

In [ ]:
!wget https://github.com/ACS-IC-labs/IC-labs/raw/main/labs/lab07/lab7.zip
!unzip -o lab7.zip

## Exercițiul 1: Analiza datelor de leakage (2p)

Încărcam date de leakage generate anterior:
 * `M`: vector de plaintexts, de lungime 50,000
 * `X`: vector de trace-uri de leakage, de lungime 50,000
 * `K`: cheia folosita pentru toate trace-urile (pentru verificare)

In [ ]:
# Load previously generated data
data = np.load("simdata.npy", allow_pickle=True).item()
M, X, K = data["M"].reshape(-1), data["X"].reshape(-1), data["K"].item()

# Get number of leakage points/plaintexts
N = X.shape[0]

print("Size of M:", M.shape)
print("Size of X:", X.shape)
print("K:", K)  # This is supposed to be found by you

Pentru modelul de leakage vom folosi Hamming weight, iar posibilele valori

*   List item
*   List item

candidat for fi intre 0 și 255.

In [ ]:
# Set possible candidate values from 0-255
target_values = np.arange(256)
nr_values = target_values.shape[0]

# Set Hamming weight as leakage model for each value in simulated data
lmodel = hamming_weight(target_values)

**TODO 1a:** Reprezentați grafic primele 1000 de valori de leakage.

In [ ]:
plt.figure(figsize=(15, 5))
idx = np.arange(1000)  # x-axis
X1 = X[idx]  # y-axis
plt.plot(idx, X1)
plt.xlabel("Sample index")
plt.ylabel("Leakage")
plt.show()

**TODO 1b:** Calculați hamming weight pentru outputului S-box-ului pentru prima valoare posibilă a cheii.

In [ ]:
k = 0                                           # The key hypothesis (i.e., the first key)
V = s_box[np.bitwise_xor(target_values[k], M)]  # The output of the S-box, on the first key
L = lmodel[V]                                   # The Hamming Weight model

**TODO 1c:** Reprezentați grafic leakage-ul hamming weight pentru outputul S-box-ului pe cheia k = 0

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(idx, L[idx])
plt.xlabel("Sample index")
plt.ylabel("Hamming weight leakage for k=%d" % k)
plt.show()

**TODO 1d:** Calculați corelația pentru această cheia dată.

In [ ]:
c = np.corrcoef(X, L)
c = c[0, 1]
print("Correlation coefficient is: %f\n" % c)

## Exercițiul 2: Determinarea cheii (4p)

Calculați corelația pentru toate valorile posibile din cheie și plotați rezultatul corelației pentru fiecare cheie posibilă.
Mai exact, folosiți comanda `plt.plot(x, y)` din Matplotlib, unde x este un vector ce reprezintă toate valorile posibile pentru cheie [0:255] și y este un vector ce conține corelația pentru fiecare cheie.

**TODO 2a:** Calculați corelația pentru fiecare posibilă cheie candidat.

În NumPy, puteți inițializa un vector astfel:
``` python
v = np.zeros(N)  # vector with N elements, initializing the elements with 0
v = np.empty(N)  # vector with N elements, without initializing the entries
```

> __Notă:__ Corelația poate fi obținută folosind [`np.corrcoef`](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html), care va returna o matrice de dimensiune 2×2 pentru cazul nostru. În acest caz, poate fi selectat elementul de la indexul (0, 1) pentru restul calculelor. Valorile de la indexul (0, 0) și indexul (1, 1) reprezintă varianța pentru fiecare variabilă (hamming weight și datele de leakage). Puteți deduce cine este elementul de la indicele (1, 0).

In [ ]:
cv = np.zeros(nr_values)  # vector with N elements
for k in range(nr_values):
    # TODO: implement your code here
    V = s_box[np.bitwise_xor(k, M)] # valoarea s_box
    L = lmodel[V] # estimarea modelului
    c = np.corrcoef(X, L) # valoarea corelației
    cv[k] = c[0, 1] # salvare valori c[0, 1] in vectorul cv


**TODO 2b:** Reprezenați grafic coeficientul de corelație pentru fiecare candidat

In [ ]:
# TODO: plot correlation coefficient for each candidate
plt.figure(figsize=(15, 5))
plt.plot(target_values, cv) # valorile target_values si corelatiile salvate
plt.xlabel("target_values")
plt.ylabel("cv")
plt.show()

## Exercițiul 3: Rata de success pentru atac (4p)

Pentru evaluarea securității unui device împotriva atacurilor de tip side-channel, una dintre metodele cele mai folosite este success rate (SR), care măsoară cât de eficient este un atac. Pentru implementarea acestei metode folosiți următorii pași:

1. Rulați atacul pe diferite seturi de trace-uri (ex. R=50 de subseturi cu N trace-uri fiecare) și pentru fiecare atac determinați dacă cheia corectă (K) conduce la cea mai mare corelație.
2. Calculați success rate ca numărul de experimente în care cheia corectă conduce la corelație maxima împărțit la numărul total de experimente. $$SR = \frac{\#cheia\ corectă\ prima}{\#experimente}$$

**TODO 3a:** Calculați rata de success pentru diferite lungimi de trace-uri folosite in atac.

> **Notă:** Pentru aceasta, folosiți lungimi variabile de trace-uri (e.g. 10, 20, 50, 100, 200, 500, 1000) și pentru fiecare iterație (să zicem 50), selectați acel număr de trace-uri în mod aleatoriu din întregul set de date.

In [ ]:
n_iter = 50
# ntraces = 100  # This should be variable (e.g., 10, 20, 50, ...)
traces = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]

rng = np.random.default_rng()

success_count = 0
success_rates = []

for ntraces in traces:
    for i in range(n_iter):
        sel_idx = rng.choice(N, ntraces)
        Mi = M[sel_idx]
        Xi = X[sel_idx]

        # TODO: obtain correlation vector for each selection of traces,
        # then compute success rate

        cv = np.zeros(nr_values)

        for k in range(nr_values):
            V = s_box[np.bitwise_xor(k, Mi)] # valoarea s_box
            L = lmodel[V] # estimarea modelului
            c = np.corrcoef(Xi, L) # valoarea corelației
            cv[k] = c[0, 1] # salvare valori c[0, 1] in vectorul cv

        best_candidate = np.argmax(cv) # indicele valorii maxime
        if best_candidate == 208: # verificare ca este egal cu cheia (k=208)
            success_count += 1
    success_rates.append(success_count/n_iter)
    success_count = 0


**TODO 3b:** Reprezentați grafic rata de success, în raport cu numărul de trace-uri.

In [ ]:
# TODO: plot success rate as a function of number of traces used in attack
plt.figure(figsize=(15, 5))
plt.plot(traces, success_rates, marker = 'o')
plt.xlabel("Number of traces")
plt.ylabel("Success rate")
plt.show()